In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pro_data

# 分支结构的特征3branch.csv 特征
    1. 分支机构的数量， BRANCH_CNT
    2. 分支结构在本省的数量, BRANCH_CNT_INHOME
    3. 分支结构在外省的数量, BRANCH_CNT_OUTHOME
    4. 第一个分支机构成立时间与总部成立时间的差（单位月）,BRANCH_FIRST_YEAR_RGYEAR_DIFF
    4.1 第一个分支机构成立时间与总部成立时间, BRANCH_FIRST_YEAR
    5. 最后一个分支机构成立时间与总部成立时间的差（单位月）, BRANCH_END_YEAR_RGYEAR_DIFF
    5.1 最后一个分支机构成立时间与总部成立时间, BRANCH_END_YEAR
    6. 平均每几个月成立一个分支结构，（没有分支结构用-1表示）, PER_BRANCH_MONTH= 最后一个分支机构成立时间与总部成立时间的差/分支机构的数量     
    7. 分支结构关停的个数，BRANCH_CLOSE_CNT
    8. 分支结构关停的比例，(关停个数/总数), BRANCH_CLOSE_RATE
    9. 分支结构正常运行的个数，BRANCH_OPEN_CNT
    10. 分支结构正常运行的比例，(运行个数/总数), BRANCH_OPEN_RATE
    11. 第一个分支机构关停时间距离总部注册时间的差值，BRANCH_FIRST_CLOSE_YEAR_RGYEAR_DIFF
    11.1  第一个分支机构关停时间 BRANCH_FIRST_CLOSE_YEAR
    12. 第一个分支机构关停时间距离第一个分支机构成立时间的差值，BRANCH_FIRST_CLOSE_YEAR_FIRST_DIFF
    13. 第一个分支机构关停时间距离最后一个分支机构成立时间的差值，BRANCH_FIRST_CLOSE_YEAR_END_DIFF

-----------------------------------------------------------------------------------------------
    14. 分支结构的数量占所有分支机构平均值的比例, BRANCH_CNT_ALL_RATE = BRANCH_CNT / 所有分支数据的平均值
    15. 企业所在大类HY的对应的总分支机构的平均值，BRANCH_HY_AVG
    16. 企业所在大类HY的对应的分支机构的平均值占所有机构的平均值的比例，BRANCH_HY_ALL_AVG_RATE
    
    17. 分支结构的关停数量占所有关停分支机构平均值的比例, BRANCH_CLOSE_CNT_ALL_RATE = BRANCH_CLOSE_CNT / 所有关停分支数据的平均值
    18. 企业所在大类HY的对应的关停分支机构的平均值，BRANCH_CLOSE_HY_AVG
    19. 企业所在大类HY的对应的关停分支机构的平均值占所有关停机构的平均值的比例，BRANCH_HY_CLOSE_ALL_AVG_RATE
    
    17. 分支结构的营业数量占所有营业分支机构平均值的比例, BRANCH_OPEN_CNT_ALL_RATE = BRANCH_OPEN_CNT / 所有营业分支数据的平均值
    18. 企业所在大类HY的对应的营业分支机构的平均值，BRANCH_OPEN_HY_AVG
    19. 企业所在大类HY的对应的营业分支机构的平均值占所有营业机构的平均值的比例，BRANCH_HY_OPEN_ALL_AVG_RATE
    
    
    20. 平均几个月关闭一个分支结构，PER_BRANCH_CLOSE_MONTH = BRANCH_CLOSE_CNT / BRANCH_END_YEAR_RGYEAR_DIFF 
    




In [33]:
df_all = pd.read_csv("../data/alldata/df_data12.csv")
df_branch = pd.read_csv("../data/public/3branch.csv")

In [34]:
df_branch.head()

,EID,TYPECODE,IFHOME,B_REYEAR,B_ENDYEAR
0,405460,br120022,0,1993,2008.0
1,138728,br210454,1,2002,NaN
2,405460,br60051,0,2011,NaN
3,247652,br210455,1,2014,NaN
4,26268,br30155,0,2014,NaN


In [35]:
# 正在运行的企业，用2018.0填充
df_branch['B_ENDYEAR'] = df_branch['B_ENDYEAR'].fillna(2018.0)

columns = df_branch.columns
df_xbranch = pd.DataFrame(columns=columns)

In [36]:
df_all.head()

,EID,RGYEAR,HY,ZCZB,ETYPE,MPNUM,INUM,FINZB,FSTINUM,TZINUM,...,CHANGE_FIRST_ALTERNO,CHANGE_END_ALTERNO,CHANGE_ALTERNO_MUCNID,FIRST_CHANGE_TIME_DIFF,END_CHANGE_TIME_DIFF,END_FIRST_CHANGE_TIME_DIFF,CHANGE_PRE_CNT,CHANGE_CNT_RATE,CHANGE_HY_CNT_AVG,CHANGE_HY_CNT_ALL_AVG
0,32741,2000-01,87,100.0,7,1.0,3.0,0.0,2.0,0.0,...,12,12,12,182,182,0,0.015744,0.924004,2.865397,0.924004
1,294220,2003-01,51,50.0,7,0.0,3.0,0.0,0.0,0.0,...,12,12,12,146,146,0,0.020849,0.981587,3.043965,0.981587
2,10526,2013-01,75,100.0,7,1.0,2.0,0.0,1.0,0.0,...,12,12,12,26,26,0,0.124123,1.040675,3.227200,1.040675
3,237382,2014-01,75,9900.0,7,3.0,4.0,0.0,2.0,0.0,...,12,12,12,14,14,0,0.230514,1.040675,3.227200,1.040675
4,473535,2008-01,75,50.0,7,3.0,5.0,0.0,1.0,0.0,...,12,12,12,86,86,0,0.037526,1.040675,3.227200,1.040675


In [37]:
EIDS = set(df_branch['EID'])

len(EIDS)

18922

In [38]:
k = 0
for EID in EIDS:
    if k%3000 == 0:
        print('第%d次处理--------->',k)
    k+=1
    tmp = df_branch[df_branch['EID'] == EID]
    row = [EID,tmp['TYPECODE'].values,tmp['IFHOME'].values,tmp['B_REYEAR'].values,tmp['B_ENDYEAR'].values]
    
    df_xbranch = df_xbranch.append(pd.Series(row,columns),ignore_index=True)
    

第%d次处理---------> 0
第%d次处理---------> 3000
第%d次处理---------> 6000
第%d次处理---------> 9000
第%d次处理---------> 12000
第%d次处理---------> 15000
第%d次处理---------> 18000


In [39]:
df_xbranch.head()

,EID,TYPECODE,IFHOME,B_REYEAR,B_ENDYEAR
0,262144.0,[br231328],[1],[2004],[2018.0]
1,6.0,[br240302],[0],[2008],[2018.0]
2,458761.0,[br239767],[1],[2004],[2018.0]
3,262154.0,[br211107],[1],[2005],[2018.0]
4,11.0,[br81227],[0],[2010],[2018.0]


In [40]:
def TotalCnt(x):
    return len(x)

df_xbranch['BRANCH_CNT'] = df_xbranch['TYPECODE'].apply(TotalCnt)


In [41]:
def InHome(x):
    return list(x).count(1)

def OutHome(x):
    return list(x).count(0)

df_xbranch['BRANCH_CNT_INHOME'] = df_xbranch['IFHOME'].apply(InHome)
df_xbranch['BRANCH_CNT_OUTHOME'] = df_xbranch['IFHOME'].apply(OutHome)


In [42]:
def getFirstTime(x):
    return str(x[0])+'-01'

def getEndTime(x):
    return str(x[-1])+'-01'

df_xbranch['BRANCH_FIRST_YEAR'] = df_xbranch['B_REYEAR'].apply(getFirstTime)
df_xbranch['BRANCH_END_YEAR'] = df_xbranch['B_REYEAR'].apply(getEndTime)



In [43]:
def EndCnt(x):
    k = 0
    for d in x:
        if d != 2018.0:
            k+=1
    return k

def OpenCnt(x):
    k = 0
    for d in x:
        if d == 2018.0:
            k+=1
    return k

df_xbranch['BRANCH_CLOSE_CNT'] = df_xbranch['B_ENDYEAR'].apply(EndCnt)
df_xbranch['BRANCH_OPEN_CNT'] = df_xbranch['B_ENDYEAR'].apply(OpenCnt)

df_xbranch['BRANCH_CLOSE_RATE'] = df_xbranch['BRANCH_CLOSE_CNT'] / df_xbranch['BRANCH_CNT']
df_xbranch['BRANCH_OPEN_RATE'] = df_xbranch['BRANCH_OPEN_CNT'] / df_xbranch['BRANCH_CNT']


In [44]:
# 倒闭的情况下才能计算
def FirstClostYear(x):
    list(x).sort()
    return str(int(x[0]))+'-01'

df_xbranch['BRANCH_FIRST_CLOSE_YEAR'] = df_xbranch['B_ENDYEAR'].apply(FirstClostYear)

In [45]:
df_xbranch.head()

,EID,TYPECODE,IFHOME,B_REYEAR,B_ENDYEAR,BRANCH_CNT,BRANCH_CNT_INHOME,BRANCH_CNT_OUTHOME,BRANCH_FIRST_YEAR,BRANCH_END_YEAR,BRANCH_CLOSE_CNT,BRANCH_OPEN_CNT,BRANCH_CLOSE_RATE,BRANCH_OPEN_RATE,BRANCH_FIRST_CLOSE_YEAR
0,262144.0,[br231328],[1],[2004],[2018.0],1,1,0,2004-01,2004-01,0,1,0.0,1.0,2018-01
1,6.0,[br240302],[0],[2008],[2018.0],1,0,1,2008-01,2008-01,0,1,0.0,1.0,2018-01
2,458761.0,[br239767],[1],[2004],[2018.0],1,1,0,2004-01,2004-01,0,1,0.0,1.0,2018-01
3,262154.0,[br211107],[1],[2005],[2018.0],1,1,0,2005-01,2005-01,0,1,0.0,1.0,2018-01
4,11.0,[br81227],[0],[2010],[2018.0],1,0,1,2010-01,2010-01,0,1,0.0,1.0,2018-01


In [46]:
df_xbranch.to_csv('../data/public/3branch_1.csv',index=False,index_label=False)
df_xbranch.columns

Index(['EID', 'TYPECODE', 'IFHOME', 'B_REYEAR', 'B_ENDYEAR', 'BRANCH_CNT',
       'BRANCH_CNT_INHOME', 'BRANCH_CNT_OUTHOME', 'BRANCH_FIRST_YEAR',
       'BRANCH_END_YEAR', 'BRANCH_CLOSE_CNT', 'BRANCH_OPEN_CNT',
       'BRANCH_CLOSE_RATE', 'BRANCH_OPEN_RATE', 'BRANCH_FIRST_CLOSE_YEAR'],
      dtype='object')

In [47]:
df_all = pd.merge(df_all,df_xbranch[['EID', 'BRANCH_CNT','BRANCH_CNT_INHOME', 'BRANCH_CNT_OUTHOME', 'BRANCH_FIRST_YEAR',
                                    'BRANCH_END_YEAR', 'BRANCH_CLOSE_CNT', 'BRANCH_OPEN_CNT','BRANCH_CLOSE_RATE', 
                                    'BRANCH_OPEN_RATE', 'BRANCH_FIRST_CLOSE_YEAR']],how='left',on=['EID'])


In [48]:
df_all.head()

,EID,RGYEAR,HY,ZCZB,ETYPE,MPNUM,INUM,FINZB,FSTINUM,TZINUM,...,BRANCH_CNT,BRANCH_CNT_INHOME,BRANCH_CNT_OUTHOME,BRANCH_FIRST_YEAR,BRANCH_END_YEAR,BRANCH_CLOSE_CNT,BRANCH_OPEN_CNT,BRANCH_CLOSE_RATE,BRANCH_OPEN_RATE,BRANCH_FIRST_CLOSE_YEAR
0,32741,2000-01,87,100.0,7,1.0,3.0,0.0,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,294220,2003-01,51,50.0,7,0.0,3.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10526,2013-01,75,100.0,7,1.0,2.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,237382,2014-01,75,9900.0,7,3.0,4.0,0.0,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,473535,2008-01,75,50.0,7,3.0,5.0,0.0,1.0,0.0,...,1.0,0.0,1.0,2009-01,2009-01,1.0,0.0,1.0,0.0,2012-01


In [49]:
# 空值填充，根据HY的类别的平均值或者众数进行填充
# ['BRANCH_CNT','BRANCH_CNT_INHOME', 'BRANCH_CNT_OUTHOME','BRANCH_CLOSE_CNT','BRANCH_OPEN_CNT', 'BRANCH_CLOSE_RATE', 'BRANCH_OPEN_RATE']使用同一个大类别的均值进行填充
# ['BRANCH_FIRST_YEAR','BRANCH_END_YEAR','BRANCH_FIRST_CLOSE_YEAR']使用同一个大类别的众数进行填充



HYLIST = set(df_all['HY'])

print(HYLIST)

meanlist = ['BRANCH_CNT','BRANCH_CNT_INHOME', 'BRANCH_CNT_OUTHOME','BRANCH_CLOSE_CNT','BRANCH_OPEN_CNT', 'BRANCH_CLOSE_RATE', 'BRANCH_OPEN_RATE']
modelist = ['BRANCH_FIRST_YEAR','BRANCH_END_YEAR','BRANCH_FIRST_CLOSE_YEAR']


{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 93}


In [50]:
for HY in HYLIST:
    # print(df_train['HY'].value_counts())
    for d in meanlist:
        df_all.loc[df_all[df_all[d].isnull()][df_all['HY']==HY].index,d] = df_all[df_all['HY']==HY][d].mean()
    
    for c in modelist:
        if(len(df_all[df_all['HY']==HY][c].value_counts().index)==0): continue
        df_all.loc[df_all[df_all[c].isnull()][df_all['HY']==HY].index,c] = df_all[df_all['HY']==HY][c].value_counts().index[0]


/home/xiaoran/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/xiaoran/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [20]:
# 测试得到，当两个文件的重复的个数超过70%以上才能进行多合并，否则就进行根文件1的单合并

# df_xalter = pd.read_csv('../data/public/2alter_1.csv')
# df_tmp = df_xalter[df_xalter['EID'].isin(df_xbranch['EID'])]
# len(df_tmp)


In [21]:
len(df_xbranch)

18922

In [51]:
# 所有关于年的空值使用注册时间填充

df_all.loc[df_all['BRANCH_FIRST_YEAR'].isnull(),'BRANCH_FIRST_YEAR'] = df_all.loc[df_all['BRANCH_FIRST_YEAR'].isnull(),'RGYEAR']
df_all.loc[df_all['BRANCH_END_YEAR'].isnull(),'BRANCH_END_YEAR'] = df_all.loc[df_all['BRANCH_END_YEAR'].isnull(),'RGYEAR']
df_all.loc[df_all['BRANCH_FIRST_CLOSE_YEAR'].isnull(),'BRANCH_FIRST_CLOSE_YEAR'] = df_all.loc[df_all['BRANCH_FIRST_CLOSE_YEAR'].isnull(),'RGYEAR']


In [52]:
df_all.head()

,EID,RGYEAR,HY,ZCZB,ETYPE,MPNUM,INUM,FINZB,FSTINUM,TZINUM,...,BRANCH_CNT,BRANCH_CNT_INHOME,BRANCH_CNT_OUTHOME,BRANCH_FIRST_YEAR,BRANCH_END_YEAR,BRANCH_CLOSE_CNT,BRANCH_OPEN_CNT,BRANCH_CLOSE_RATE,BRANCH_OPEN_RATE,BRANCH_FIRST_CLOSE_YEAR
0,32741,2000-01,87,100.0,7,1.0,3.0,0.0,2.0,0.0,...,1.669714,0.836571,0.833143,2014-01,2013-01,0.748571,0.921143,0.431498,0.568502,2018-01
1,294220,2003-01,51,50.0,7,0.0,3.0,0.0,0.0,0.0,...,12.798897,2.308456,10.490441,2004-01,2004-01,1.439706,11.359191,0.496616,0.503384,2018-01
2,10526,2013-01,75,100.0,7,1.0,2.0,0.0,1.0,0.0,...,2.083667,0.919283,1.164384,2005-01,2004-01,0.984826,1.098841,0.483373,0.516627,2018-01
3,237382,2014-01,75,9900.0,7,3.0,4.0,0.0,2.0,0.0,...,2.083667,0.919283,1.164384,2005-01,2004-01,0.984826,1.098841,0.483373,0.516627,2018-01
4,473535,2008-01,75,50.0,7,3.0,5.0,0.0,1.0,0.0,...,1.000000,0.000000,1.000000,2009-01,2009-01,1.000000,0.000000,1.000000,0.000000,2012-01


In [53]:
def timeDiff(x):
    a = x[:x.find(':')]
    b = x[x.find(':')+1:]
    y = int(a[:a.find('-')]) - int(b[:b.find('-')])
    m = int(a[a.find('-')+1:]) - int(b[b.find('-')+1:])
    return y * 12 + m
    
df_all['BRANCH_FIRST_YEAR_RGYEAR_DIFF'] = (df_all['BRANCH_FIRST_YEAR'] + ':' + df_all['RGYEAR']).apply(timeDiff)
df_all['BRANCH_END_YEAR_RGYEAR_DIFF'] = (df_all['BRANCH_END_YEAR'] + ':' + df_all['RGYEAR']).apply(timeDiff)

df_all['BRANCH_FIRST_CLOSE_YEAR_FIRST_DIFF'] = (df_all['BRANCH_FIRST_CLOSE_YEAR'] + ':' + df_all['BRANCH_FIRST_YEAR']).apply(timeDiff)
df_all['BRANCH_FIRST_CLOSE_YEAR_END_DIFF'] = (df_all['BRANCH_FIRST_CLOSE_YEAR'] + ':' + df_all['BRANCH_END_YEAR']).apply(timeDiff)



In [54]:
df_all['PER_BRANCH_MONTH'] = df_all['BRANCH_END_YEAR_RGYEAR_DIFF'] / df_all['BRANCH_CNT']
df_all['PER_BRANCH_CLOSE_MONTH'] = df_all['BRANCH_END_YEAR_RGYEAR_DIFF'] / df_all['BRANCH_CLOSE_CNT']



In [55]:
df_all['BRANCH_CNT_ALL_RATE'] = df_all['BRANCH_CNT'] / df_all['BRANCH_CNT'].mean()

df_all['BRANCH_CLOSE_CNT_ALL_RATE']  = df_all['BRANCH_CLOSE_CNT'] / df_all['BRANCH_CLOSE_CNT'].mean()

df_all['BRANCH_OPEN_CNT_ALL_RATE'] = df_all['BRANCH_OPEN_CNT'] / df_all['BRANCH_OPEN_CNT'].mean()


In [56]:
tmp = pd.DataFrame()

tmp['BRANCH_HY_AVG'] = df_all.groupby(['HY'],as_index=True,axis=0)['BRANCH_CNT'].mean()
tmp['BRANCH_HY_ALL_AVG_RATE'] = tmp['BRANCH_HY_AVG'] / df_all['BRANCH_CNT'].mean()

tmp['BRANCH_CLOSE_HY_AVG'] = df_all.groupby(['HY'],as_index=True,axis=0)['BRANCH_CLOSE_CNT'].mean()
tmp['BRANCH_HY_CLOSE_ALL_AVG_RATE'] = tmp['BRANCH_CLOSE_HY_AVG'] / df_all['BRANCH_CLOSE_CNT'].mean()

tmp['BRANCH_OPEN_HY_AVG'] = df_all.groupby(['HY'],as_index=True,axis=0)['BRANCH_OPEN_CNT'].mean()
tmp['BRANCH_HY_OPEN_ALL_AVG_RATE'] = tmp['BRANCH_OPEN_HY_AVG'] / df_all['BRANCH_OPEN_CNT'].mean()


tmp['HY'] = tmp.index

df_all = pd.merge(df_all,tmp,how='left',on=['HY'])


In [57]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255130 entries, 0 to 255129
Data columns (total 73 columns):
EID                                   255130 non-null int64
RGYEAR                                255130 non-null object
HY                                    255130 non-null int64
ZCZB                                  255130 non-null float64
ETYPE                                 255130 non-null int64
MPNUM                                 255130 non-null float64
INUM                                  255130 non-null float64
FINZB                                 255130 non-null float64
FSTINUM                               255130 non-null float64
TZINUM                                255130 non-null float64
HY_ZCZB_AVG                           255130 non-null float64
HY_ZCZB_AVG_RATE                      255130 non-null float64
MPNUM_HY_AVG                          255130 non-null float64
INUM_HY_AVG                           255130 non-null float64
FINZB_HY_AVG                

In [58]:
# 得到在df_xchange的所有数据，
df_all = df_all.fillna(0)

df_all.to_csv('../data/alldata/df_data123.csv',index=False,index_label=False)

In [74]:
len(df_xall)

18922